In [1]:
from parameters import *
from functions import *
from PDE_solutions import *
import time
walks = 500
N = 200

In [2]:
X0

[-1.5, 1.0]

# MLMV



## With different starting positions

In [3]:
#defines variables for the MLMC with different starting positions

In [3]:
L = 4
N = 200

X0 = np.c_[np.linspace(-1, -1.5,L+1), np.linspace(0,1,L+1)]
Walks = np.linspace(2000,100, L+1, dtype=int).tolist()

In [4]:
Walks

[2000, 1525, 1050, 575, 100]

In [73]:
#runs a simulation for the points and prints out the time
BasicMonteCarlo(X0[L], Walks[L], N)

Current walk:  0

Number of simulations: 100. Time needed = 0.61 s
Estimated variance: 0.47609522856952335
The estimated probability at [-1.5  1. ] is: 0.66 (using MC)
Confidence interval: [ 0.66 +- 0.09446762230600492 ]	with P = 0.95%


(0.66, 0.47609522856952335, 0.5655323776939951, 0.754467622306005)

In [75]:
#runs the MLMC
Functions = MultilevelFunctionForLDifferentPositions
E, VAR, Var, mean, VarFromTheLwalk = MultiLevelMonteCarlo(L,X0,Walks, Functions, N)

Calculating level 4
Calculating level 3
Calculating level 2
Calculating level 1
Calculating level 0

Number of simulations: 5250. Time needed = 5.20 s
The estimated probability at [[-1.     0.   ]
 [-1.125  0.25 ]
 [-1.25   0.5  ]
 [-1.375  0.75 ]
 [-1.5    1.   ]] is: 0.7103895903336388 (using MC)
with the variance :  0.0044100066514801625
Whithout the variance reduction  0.004688261722621507


In [4]:
E, VAR, Var, mean, VarFromTheLwalk

(array([ 0.718, -0.018, -0.005]),
 array([0.45 , 0.244, 0.158]),
 0.0012616087912976682,
 0.695,
 0.002262736147758253)

# With different timestep each level

In [37]:
#Defines parameters
L = 5
X0 = np.array([-1.5,1])

N0 = 99
NL = 100
N = createVectorN(N0,NL,L)

#Walks = [ max(round(1000 * 2 **-i),100) for i in range(L+1)]
#Walks = [1000- 300*i for i in range(L+1)]
Walks = [round(2000*2**-i) for i in range(L+1)]
print(Walks)

[2000, 1000, 500, 250, 125, 62]


In [140]:
#runs a simulation for the points and prints out the time
BasicMonteCarlo(X0, Walks[L], N[L])

Current walk:  0

Number of simulations: 100. Time needed = 1.38 s
Estimated variance: 0.4605661864718383
The estimated probability at [-1.5  1. ] is: 0.7 (using MC)
Confidence interval: [ 0.7 +- 0.09138632344891306 ]	with P = 0.95%


(0.7, 0.4605661864718383, 0.6086136765510869, 0.791386323448913)

In [38]:
Function = MultilevelFunctionForLDifferentTimesSteps
E, VAR, Var, mean, VarFromTheLwalk = MultiLevelMonteCarlo(L,X0,Walks, Function, N)

Calculating level 5
Calculating level 4
Calculating level 3
Calculating level 2
Calculating level 1
Calculating level 0

Number of simulations: 3937. Time needed = 7.20 s
The estimated probability at [-1.5  1. ] is: 0.665 (using MC)
with the variance :  0.0004883275839637458
Whithout the variance reduction  0.007254011935599251


In [30]:
VAR

array([0.461, 0.045])

In [ ]:
#run with small levels

In [43]:
MLMCVariances = []
VariancesOfTheNaive = []
ExecutionsTimes = []
Means = []

for L in range(1,10):
    for N0 in [1,10,100, 1000, 9000]:
        NL = 10000
        N = createVectorN(N0,NL,L)

        #Walks = [ max(round(1000 * 2 **-i),100) for i in range(L+1)]
        #Walks = [1000- 300*i for i in range(L+1)]
        Walks = [round(10000*2**-i) for i in range(L+1)]
        print(Walks, N0)
        Function = MultilevelFunctionForLDifferentTimesSteps
        
        startingtime = time.time()
        #proceeds to the walk
        E, VAR, Var, mean, VarFromTheLwalk = MultiLevelMonteCarlo(L,X0,Walks, Function, N)
        
        #Stores the values
        ExecutionsTimes.append(time.time() - startingtime)
        VariancesOfTheNaive.append(VarFromTheLwalk)
        MLMCVariances.append(Var)
        Means.append(mean)

[10000, 5000] 1
Calculating level 1
Calculating level 0

Number of simulations: 15000. Time needed = 3115.97 s
The estimated probability at [-1.5  1. ] is: 0.7156 (using MC)
with the variance :  9.434100170759684e-05
Whithout the variance reduction  9.011955784923376e-05
[10000, 5000] 10
Calculating level 1
Calculating level 0

Number of simulations: 15000. Time needed = 2071.60 s
The estimated probability at [-1.5  1. ] is: 0.7086000000000001 (using MC)
with the variance :  0.0001817371002855432
Whithout the variance reduction  9.053777674431575e-05
[10000, 5000] 100
Calculating level 1
Calculating level 0

Number of simulations: 15000. Time needed = 1879.97 s
The estimated probability at [-1.5  1. ] is: 0.7246 (using MC)
with the variance :  0.00017125871533703224
Whithout the variance reduction  8.900687863805235e-05
[10000, 5000] 1000
Calculating level 1
Calculating level 0

Number of simulations: 15000. Time needed = 1962.45 s
The estimated probability at [-1.5  1. ] is: 0.7128 (u

Calculating level 5
Calculating level 4
Calculating level 3
Calculating level 2
Calculating level 1
Calculating level 0

Number of simulations: 19843. Time needed = 129.94 s
The estimated probability at [-1.5  1. ] is: 0.6953999999999999 (using MC)
with the variance :  0.006795488171830458
Whithout the variance reduction  0.002968108343947343
[10000, 5000, 2500, 1250, 625, 312, 156] 10
Calculating level 6
Calculating level 5
Calculating level 4
Calculating level 3
Calculating level 2
Calculating level 1
Calculating level 0

Number of simulations: 19843. Time needed = 143.66 s
The estimated probability at [-1.5  1. ] is: 0.6689487179487181 (using MC)
with the variance :  0.006139199956245792
Whithout the variance reduction  0.0028524912720281174
[10000, 5000, 2500, 1250, 625, 312, 156] 100
Calculating level 6
Calculating level 5
Calculating level 4
Calculating level 3
Calculating level 2
Calculating level 1
Calculating level 0

Number of simulations: 19843. Time needed = 241.44 s
The es

Calculating level 8
Calculating level 7
Calculating level 6
Calculating level 5
Calculating level 4
Calculating level 3
Calculating level 2
Calculating level 1
Calculating level 0

Number of simulations: 19980. Time needed = 3199.45 s
The estimated probability at [-1.5  1. ] is: 0.7178153846153846 (using MC)
with the variance :  0.0009012161126075206
Whithout the variance reduction  0.02051956704170308


In [44]:
ExecutionsTimes

[3115.975056409836,
 2071.5975263118744,
 1879.9751732349396,
 1962.4459800720215,
 3080.504043340683,
 1428.8712241649628,
 1162.7978763580322,
 1132.6685433387756,
 1441.92165184021,
 3136.3314156532288,
 752.8857479095459,
 659.2707724571228,
 714.5596270561218,
 1124.3720834255219,
 3165.4027087688446,
 412.1994459629059,
 389.06049966812134,
 480.22207021713257,
 909.7376165390015,
 3196.4724671840668,
 225.52961897850037,
 236.4859104156494,
 335.87942123413086,
 784.7279188632965,
 3189.3890750408173,
 129.94210267066956,
 143.66137957572937,
 241.44104862213135,
 669.6496036052704,
 3158.533038377762,
 77.42936563491821,
 95.00698399543762,
 183.95114159584045,
 619.8259789943695,
 3200.120740175247,
 45.605005979537964,
 64.42817878723145,
 147.6716067790985,
 567.7597961425781,
 3215.957525253296,
 28.622167348861694,
 47.97873306274414,
 120.68338775634766,
 527.9357132911682,
 3199.451230764389]

In [45]:
VariancesOfTheNaive

[9.011955784923376e-05,
 9.053777674431575e-05,
 8.900687863805235e-05,
 9.046251761596448e-05,
 9.063136548062032e-05,
 0.00018168891530519386,
 0.00018333969940564226,
 0.0001786011000558968,
 0.00018333969940564226,
 0.00018041080683731085,
 0.00036027766180388686,
 0.00035996747473388907,
 0.0003627089254646063,
 0.0003630066068783182,
 0.00036619125935874626,
 0.0007376480989934957,
 0.0007298202096828633,
 0.0007226889957757657,
 0.0007286590423510848,
 0.0007202233271879224,
 0.0014395899291479578,
 0.0014684239057798267,
 0.0014900740072579229,
 0.0014544059962686628,
 0.0014017704493361788,
 0.002968108343947343,
 0.0028524912720281174,
 0.002984950834738278,
 0.0025017742192645566,
 0.002852491272028118,
 0.006021721592342772,
 0.005883801276330986,
 0.006021721592342771,
 0.005723475242616447,
 0.006189801917759841,
 0.009372250466543478,
 0.012637539951029335,
 0.011989023079987641,
 0.012245305972844083,
 0.01134253923217325,
 0.02051956704170308,
 0.02350811729908136,
 0.

In [46]:
MLMCVariances

[9.434100170759684e-05,
 0.0001817371002855432,
 0.00017125871533703224,
 0.00015898211768083337,
 8.509285624918111e-05,
 0.0003485662138046207,
 0.00042295653977931233,
 0.00039069730609599714,
 0.000335942561069178,
 0.00011974990182105625,
 0.0008237646068658972,
 0.000854635281356569,
 0.0007790165760902113,
 0.0006588665502410692,
 0.00020644955605822154,
 0.0017430320767811858,
 0.0016242077520301698,
 0.001530365427422453,
 0.0012206142138509806,
 0.00034816181779736703,
 0.003406815129307486,
 0.003290570085134375,
 0.002827293016810448,
 0.002503058130977889,
 0.0004938890585045631,
 0.006795488171830458,
 0.006139199956245792,
 0.0054636985977721975,
 0.004002038004805612,
 0.0011091718617790675,
 0.013487909791974725,
 0.01209346660617774,
 0.011391694316911286,
 0.006605989320198958,
 0.00028153840279788344,
 0.019342432896379106,
 0.023844264282304798,
 0.021736723677429334,
 0.017659877720313806,
 0.0002915213578789207,
 0.04536855273401037,
 0.045149284176546296,
 0.051

In [47]:
Means

[0.7156,
 0.7086000000000001,
 0.7246,
 0.7128,
 0.7147,
 0.7061000000000001,
 0.7045,
 0.7233,
 0.7121,
 0.7080000000000001,
 0.7425,
 0.7217,
 0.7211,
 0.7129,
 0.7137,
 0.6929000000000001,
 0.7041,
 0.7258,
 0.7097,
 0.7151,
 0.7291410256410258,
 0.7007846153846155,
 0.6561538461538461,
 0.7382256410256409,
 0.7160948717948717,
 0.6953999999999999,
 0.6689487179487181,
 0.7160153846153846,
 0.7788025641025642,
 0.723,
 0.7300666666666666,
 0.6960000000000001,
 0.6379025641025642,
 0.7403205128205128,
 0.7071999999999999,
 0.7569384615384617,
 0.644502564102564,
 0.5883205128205129,
 0.663128205128205,
 0.7048,
 0.899725641025641,
 0.6779487179487181,
 0.7329102564102564,
 0.6828435897435897,
 0.7178153846153846]

In [49]:
np.array(MLMCVariances) - np.array(VariancesOfTheNaive)

array([ 4.221e-06,  9.120e-05,  8.225e-05,  6.852e-05, -5.539e-06,
        1.669e-04,  2.396e-04,  2.121e-04,  1.526e-04, -6.066e-05,
        4.635e-04,  4.947e-04,  4.163e-04,  2.959e-04, -1.597e-04,
        1.005e-03,  8.944e-04,  8.077e-04,  4.920e-04, -3.721e-04,
        1.967e-03,  1.822e-03,  1.337e-03,  1.049e-03, -9.079e-04,
        3.827e-03,  3.287e-03,  2.479e-03,  1.500e-03, -1.743e-03,
        7.466e-03,  6.210e-03,  5.370e-03,  8.825e-04, -5.908e-03,
        9.970e-03,  1.121e-02,  9.748e-03,  5.415e-03, -1.105e-02,
        2.485e-02,  2.164e-02,  2.879e-02, -1.018e-02, -1.962e-02])

In [53]:
#calculates the evolution of variance for increasing number of steps

MLMCVariance = []
L=3
initialNumbersOfWalks = [100,200,500,1000,2000,5000,10000,20000,50000]
#initialNumbersOfWalks = [20,50,100]
for w in initialNumbersOfWalks :
        N0 = 1000
        NL = 10000
        N = createVectorN(N0,NL,L)

        Walks = [round(w*2**-i) for i in range(L+1)]
        print(Walks)
        Function = MultilevelFunctionForLDifferentTimesSteps
        #proceeds to the walk
        E, VAR, Var, mean, VarFromTheLwalk = MultiLevelMonteCarlo(L,X0,Walks, Function, N)
        MLMCVariance.append(Var)

[100, 50, 25, 12]
Calculating level 3
Calculating level 2
Calculating level 1
Calculating level 0

Number of simulations: 187. Time needed = 8.56 s
The estimated probability at [-1.5  1. ] is: 0.7666666666666666 (using MC)
with the variance :  0.04943172334401835
Whithout the variance reduction  0.03243745600673012
[200, 100, 50, 25]
Calculating level 3
Calculating level 2
Calculating level 1
Calculating level 0

Number of simulations: 375. Time needed = 24.54 s
The estimated probability at [-1.5  1. ] is: 0.68 (using MC)
with the variance :  0.029524565390922783
Whithout the variance reduction  0.01833030277982336
[500, 250, 125, 62]
Calculating level 3
Calculating level 2
Calculating level 1
Calculating level 0

Number of simulations: 937. Time needed = 60.08 s
The estimated probability at [-1.5  1. ] is: 0.752 (using MC)
with the variance :  0.01227820876293357
Whithout the variance reduction  0.007601288065592049
[1000, 500, 250, 125]
Calculating level 3
Calculating level 2
Calcula

KeyboardInterrupt: 

In [54]:
#calculates the evolution of variance for increasing number of steps

MLMCVariance = []
L=3
initialNumbersOfWalks = [100,200,500,1000,2000,5000,10000,20000,50000]
#initialNumbersOfWalks = [20,50,100]
for w in initialNumbersOfWalks :
        N0 = 990
        NL = 1000
        N = createVectorN(N0,NL,L)

        Walks = [round(w*2**-i) for i in range(L+1)]
        print(Walks)
        Function = MultilevelFunctionForLDifferentTimesSteps
        #proceeds to the walk
        E, VAR, Var, mean, VarFromTheLwalk = MultiLevelMonteCarlo(L,X0,Walks, Function, N)
        MLMCVariance.append(Var)

[100, 50, 25, 12]
Calculating level 3
Calculating level 2
Calculating level 1
Calculating level 0

Number of simulations: 187. Time needed = 3.53 s
The estimated probability at [-1.5  1. ] is: 0.71 (using MC)
with the variance :  0.0045604802157206865
Whithout the variance reduction  0.037688918072220454
[200, 100, 50, 25]
Calculating level 3
Calculating level 2
Calculating level 1
Calculating level 0

Number of simulations: 375. Time needed = 6.87 s
The estimated probability at [-1.5  1. ] is: 0.73 (using MC)
with the variance :  0.006079055177794011
Whithout the variance reduction  0.018330302779823362
[500, 250, 125, 62]
Calculating level 3
Calculating level 2
Calculating level 1
Calculating level 0

Number of simulations: 937. Time needed = 17.93 s
The estimated probability at [-1.5  1. ] is: 0.716 (using MC)
with the variance :  0.0016104706860628941
Whithout the variance reduction  0.007115189241242197
[1000, 500, 250, 125]
Calculating level 3
Calculating level 2
Calculating leve

In [55]:
MLMCVariance

[0.0045604802157206865,
 0.006079055177794011,
 0.0016104706860628941,
 0.00044873194711359736,
 0.0003600242899646652,
 0.00019062631812587997,
 0.00010270351635523569,
 4.458491339473772e-05,
 2.0117489600350527e-05]

In [ ]:
#calculates the evolution of variance for increasing number of steps

MLMCVariance = []
L=10
initialNumbersOfWalks = [2000,5000,10000,20000,50000,100000]
#initialNumbersOfWalks = [20,50,100]
for w in initialNumbersOfWalks :
        N0 = 100
        NL = 1000
        N = createVectorN(N0,NL,L)

        Walks = [round(w*2**-i) for i in range(L+1)]
        print(Walks)
        Function = MultilevelFunctionForLDifferentTimesSteps
        #proceeds to the walk
        E, VAR, Var, mean, VarFromTheLwalk = MultiLevelMonteCarlo(L,X0,Walks, Function, N)
        MLMCVariance.append(Var)

[2000, 1000, 500, 250, 125, 62, 31, 16, 8, 4, 2]
Calculating level 10
Calculating level 9
Calculating level 8
Calculating level 7
Calculating level 6
Calculating level 5
Calculating level 4
Calculating level 3
Calculating level 2
Calculating level 1
Calculating level 0

Number of simulations: 3998. Time needed = 12.49 s
The estimated probability at [-1.5  1. ] is: 0.529 (using MC)
with the variance :  0.08599738498576506
Whithout the variance reduction  0.3535533905932738
[5000, 2500, 1250, 625, 312, 156, 78, 39, 20, 10, 5]
Calculating level 10
Calculating level 9
Calculating level 8
Calculating level 7
Calculating level 6
